In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Importing all libraries**

In [ ]:
import nltk
import re
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# **preprocessing data**

In [ ]:
data = pd.read_csv('/kaggle/input/email-spam-ham-prediction/sms_spam.csv')
data.head()

In [ ]:
data.shape

In [ ]:
message = data['text'].values

In [ ]:
len(message)

In [ ]:
corpus = []
ps = PorterStemmer()
lemma = WordNetLemmatizer()
for i in range(len(message)):
    text = re.sub('[^a-zA-Z]',' ',message[i])
    text = text.lower()
    text = text.split()
    text = [lemma.lemmatize(word) for word in text if not word in set(stopwords.words('english'))]
    text = ' '.join(text)
    corpus.append(text)

* re.sub take care's of only text like(a-z,A-Z) is selected rather than symbols like('.',',','!')
* text.lower() is converting all capital letters if present to lowercase so if word is repeated we can find it
* text.split() it is spliting the paragraph into words
* [lemma.lemmatize(word) for word in text if not word in set(stopwords.words('english'))] is lemmatize the words and removing stopwords

In [ ]:
len(corpus)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer()
x= cv.fit_transform(corpus).toarray()

In [ ]:
len(x[0])


In [ ]:
y = pd.get_dummies(data['type'])
y = y.iloc[:,1].values
y

# **Building model**

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
accuracy = []
skf = StratifiedKFold(n_splits=10)
for train_index,test_index in skf.split(x,y):
    model = MultinomialNB()
    model.fit(x[train_index],y[train_index])
    y_pred = model.predict(x[test_index])
    accuracy.append(accuracy_score(y[test_index],y_pred))
    

In [ ]:
len(accuracy)

In [ ]:
np.array(accuracy).mean()